In [10]:
# Import dependanciies
import pandas as pd
from sqlalchemy import create_engine
import psycopg2
from Postgres_Login import username
from Postgres_Login import password

In [11]:
# Import csv file
csv_file = "Resources/Prime_TV_Shows_Dataset_JC.csv"
Raw_Prime_Data = pd.read_csv(csv_file)

In [12]:
# Select specific columns, rename selected columns and add Prime column to specify which streaming platform its on.
Focused_Prime_Data = Raw_Prime_Data[["Name of the show", "No of seasons available", "Genre", "Age of viewers"]]

Cleaned_Prime_Data = Focused_Prime_Data.rename(columns = {"Name of the show" : "Title of Show",
                                                          "No of seasons available" : "No. of Seasons",
                                                          "Genre" : "Genres",
                                                          "Age of viewers" : "TV Rating"})

Cleaned_Prime_Data = Cleaned_Prime_Data [["Title of Show", "Genres", "No. of Seasons", "TV Rating"]]

Cleaned_Prime_Data["Prime"] = "Yes"

Cleaned_Prime_Data = Cleaned_Prime_Data.dropna()

Cleaned_Prime_Data

,Title of Show,Genres,No. of Seasons,TV Rating,Prime
0,Pataal Lok,Drama,1.0,18+,Yes
1,Upload,Sci-fi comedy,1.0,16+,Yes
2,The Marvelous Mrs. Maisel,"Drama, Comedy",3.0,16+,Yes
3,Four More Shots Please,"Drama, Comedy",2.0,18+,Yes
4,Fleabag,Comedy,2.0,18+,Yes
...,...,...,...,...,...
388,Chris Tall Presents�,Comedy,1.0,16+,Yes
389,Short Poppies,Comedy,1.0,13+,Yes
390,The 2018 Rose Parade Hosted by Cord & Tish,Comedy,1.0,All,Yes
391,Aliens Love Underpants And�,Kids,1.0,All,Yes


In [13]:
# Removing "+" and renaming ages to match offical TV Rating titles.
Cleaned_Prime_Data["TV Rating"] = Cleaned_Prime_Data["TV Rating"].str.strip("+").astype(str)
Cleaned_Prime_Data["TV Rating"].replace(regex = True, inplace = True, to_replace = "18", value = "TV-MA")
Cleaned_Prime_Data["TV Rating"].replace(regex = True, inplace = True, to_replace = "16", value = "TV-MA")
Cleaned_Prime_Data["TV Rating"].replace(regex = True, inplace = True, to_replace = "13", value = "TV-14")
Cleaned_Prime_Data["TV Rating"].replace(regex = True, inplace = True, to_replace = "7", value = "TV-7")
Cleaned_Prime_Data["TV Rating"].replace(regex = True, inplace = True, to_replace = "All", value = "TV-G")
Cleaned_Prime_Data

,Title of Show,Genres,No. of Seasons,TV Rating,Prime
0,Pataal Lok,Drama,1.0,TV-MA,Yes
1,Upload,Sci-fi comedy,1.0,TV-MA,Yes
2,The Marvelous Mrs. Maisel,"Drama, Comedy",3.0,TV-MA,Yes
3,Four More Shots Please,"Drama, Comedy",2.0,TV-MA,Yes
4,Fleabag,Comedy,2.0,TV-MA,Yes
...,...,...,...,...,...
388,Chris Tall Presents�,Comedy,1.0,TV-MA,Yes
389,Short Poppies,Comedy,1.0,TV-14,Yes
390,The 2018 Rose Parade Hosted by Cord & Tish,Comedy,1.0,TV-G,Yes
391,Aliens Love Underpants And�,Kids,1.0,TV-G,Yes


In [14]:
# Created new csv of cleaned data
Cleaned_Prime_Data.to_csv("Resources/Cleaned_Prime_TV_Shows_Dataset_JC.csv")

In [15]:
# Connect to local database
rds_connection_string = f"{username}:{password}@localhost:5432/streaming_services"
engine = create_engine(f"postgresql://{rds_connection_string}")

In [16]:
# Send data to database to create a table
Cleaned_Prime_Data.to_sql(name = "prime", con = engine, if_exists = "append", index = False)

In [17]:
# Table Preview from database
pd.read_sql_query("select * from prime", con = engine).head()

,Title of Show,Genres,No. of Seasons,TV Rating,Prime
0,Pataal Lok,Drama,1.0,TV-MA,Yes
1,Upload,Sci-fi comedy,1.0,TV-MA,Yes
2,The Marvelous Mrs. Maisel,"Drama, Comedy",3.0,TV-MA,Yes
3,Four More Shots Please,"Drama, Comedy",2.0,TV-MA,Yes
4,Fleabag,Comedy,2.0,TV-MA,Yes
